# Entendiendo el problema

Como ya sabemos queremos hacer un bot automatico de respuestas cortas para algunas IM. 

Aquí vienen un par preguntas:
   - ¿Que tipo de respuestas dara nuestro bot?
   - ¿Como podemos encarar este problema de la mejor manera posible?

Para poder contestar estas preguntas hay que ver los datos que tenemos hasta ahora.

In [1]:
import pandas as pd
import csv

In [63]:
df_pre1 = pd.read_csv('preprocessed_corpus_pre1.csv')
df_pre2 = pd.read_csv('preprocessed_corpus_pre2.csv')
df_pre3 = pd.read_csv('preprocessed_corpus_pre3.csv')
df_pre4 = pd.read_csv('preprocessed_corpus_pre4.csv')

In [64]:
df_pre1.head(10)

,Unnamed: 0,id,owner,date,hour,message
0,0,1,USER0,1/27/16,12:24,MEDIA
1,1,1,USER0,2/4/16,09:50,lee esta historia ! . una mujer muy enferma...
2,2,1,USER0,4/4/16,11:28,hola dani ! vos sabes a donde venden fotocopia...
3,3,1,USER1,4/4/16,11:37,le pregunto a mi jefe
4,4,1,USER0,4/4/16,11:38,gracias ! para cuando sabes ?
5,5,1,USER0,4/4/16,11:38,💃
6,6,1,USER1,4/4/16,11:40,le pregunto ahora a ver si sabe algo : p
7,7,1,USER0,4/4/16,11:59,👏
8,8,1,USER0,4/6/16,08:46,"✳ whatsapp le comunica , . que debido a la..."
9,9,1,USER1,4/6/16,08:46,ja


In [65]:
distribution_pre1 = pd.read_csv('distribution_pre1.csv')
distribution_pre2 = pd.read_csv('distribution_pre2.csv')
distribution_pre3 = pd.read_csv('distribution_pre3.csv')
distribution_pre4 = pd.read_csv('distribution_pre4.csv')

Uno pensaría que los datos no son de mucha utilidad, sin embargo después de discutir un largo rato decidimos encarar el problema como si fuera un problema de machine learning supervisado donde el contexto de los mensajes serían los datos entrenamiento y como etiquetas las respuestas cortas.

Perfecto! Ya sabemos como encararlo... pero antes de empezar debemos aclarar un par definiciones:

   - ¿que es una respuesta corta? cualquier turno (de diálogo) hasta 3 palabras que ocurren por lo menos n veces en el corpus. Haremos varios experimentos con diferentes longitud de respuestas cortas.
   - ¿que es el contexto de una respuesta corta? El contexto de una respuesta corta son los mensajes que ocurren t turnos anteriores a una respuesta corta.

## Distribución de respuestas cortas

Veamos ahora la distribución de las respuestas cortas

In [54]:
from collections import Counter

N = 5
d = {}

counter_pre1 = Counter([' '.join(str(answer).split()) for answer in df.pre1 if 0 < len(str(answer).split()) <= N])
counter_pre2 = Counter([' '.join(str(answer).split()) for answer in df.pre2 if 0 < len(str(answer).split()) <= N])
counter_pre3 = Counter([' '.join(str(answer).split()) for answer in df.pre3 if 0 < len(str(answer).split()) <= N])
counter_pre4 = Counter([' '.join(str(answer).split()) for answer in df.pre4 if 0 < len(str(answer).split()) <= N])

In [55]:
distribution_answer_pre1 = pd.DataFrame(
    data={
        'answer': list(counter_pre1.keys()),
        'counter': list(counter_pre1.values())
    }
)

In [56]:
distribution_answer_pre2 = pd.DataFrame(
    data={
        'answer': list(counter_pre2.keys()),
        'counter': list(counter_pre2.values())
    }
)

In [57]:
distribution_answer_pre3 = pd.DataFrame(
    data={
        'answer': list(counter_pre3.keys()),
        'counter': list(counter_pre3.values())
    }
)

In [58]:
distribution_answer_pre4 = pd.DataFrame(
    data={
        'answer': list(counter_pre4.keys()),
        'counter': list(counter_pre4.values())
    }
)

In [61]:
distribution_answer_pre1.sort_values('counter', ascending=False).head(10)

,answer,counter
0,MEDIA,16066
4,ja,8857
47,ok,1197
85,😂,1191
55,si,965
38,NUM,556
303,?,525
3358,🤣,469
159,no,446
337,dale,433


In [62]:
distribution_answer_pre1.to_csv(path_or_buf='distribution_answer_pre1.csv')
distribution_answer_pre2.to_csv(path_or_buf='distribution_answer_pre2.csv')
distribution_answer_pre3.to_csv(path_or_buf='distribution_answer_pre3.csv')
distribution_answer_pre4.to_csv(path_or_buf='distribution_answer_pre4.csv')

## Etiquetando los contextos

### Respecto a una respuesta corta

Una vez definido el problema, podemos empezar a etiquetar los contextos.

Al principio diferenciaremos los etiquetados según el tamaño de respuestas: 
   - respuesta cortas hasta 1 a 3 palabras

Empezaremos a etiquetar los contextos sin importar los turnos. Para ello seguiremos un par de suposiciones:
   -  Si hay una respuesta corta:
       - Si esta en el mismo chat:
             - si no es la misma persona que escribe quiere decir que hubo una respuesta corta. Agregamos esta respuesta corta al contexto siguiente.
             - si es la misma persona que escribe la respuesta corta i.e. escribe una respuesta larga y luego una respuesta corta agregamos al contexto.
       - Si esta en diferente chat etiquetamos como nonDef debido a que no hubo una respuesta corta después de todo.
   - Si es una respuesta larga:
        - Si esta en el mismo chat se agrega al contexto
        - Si no esta, el contexto anterior lo etiquetamos como nondef

In [7]:
def is_short(answer, short_answer):
    """
    is short answer?
    """
    return len(answer.split(' ')) <= short_answer

def is_same_owner(own1, own2):
    """
    return whenever own1 is the same as own2
    """
    return own1==own2

def is_same_chat(idc1, idc2):
    """
    Is it the same chat?
    """
    return idc1 == idc2

In [5]:
fieldnames = ['idchat', 'context', 'label']

def label_the_data(filename, answer_length):
    with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        writer.writeheader()

        last_owner = ''
        last_chat = 0
        row = {'idchat': 0, 'context': '', 'label': ''}
        for actual_chat, owner, date, hour, text in df.values:
            # respuesta corta
            if is_short(text, answer_length):
                # es el mismo chat
                if is_same_chat(actual_chat, last_chat):
                    # si no es la misma persona que escribe quiere decir que hubo una respuesta (en este caso corta).
                    # el label será la respuesta en sí. Empezará un contexto nuevo de la otra persona.
                    if not is_same_owner(last_owner, owner):
                        row['label'] = text
                        writer.writerow(row)
                        row['context'] = text
                    else:
                        row['context'] += '. ' + text
                # diferente chat
                else:
                    row['label'] = 'nonDef'
                    writer.writerow(row)
                    row['context'] = text
                        
            else:
                # respuesta larga
                if is_same_chat(actual_chat, last_chat):
                    # es el mismo chat
                    row['context'] += '. ' + text
                else: 
                    # es diferente chat
                    row['label'] = 'nonDef'
                    writer.writerow(row)
                    row['context'] = text

            row['idchat'] = actual_chat
            last_owner = owner
            last_chat = actual_chat

In [6]:
label_the_data('short_ans1.csv', 1)
label_the_data('short_ans2.csv', 2)
label_the_data('short_ans3.csv', 3)

### Respecto a los turnos anteriores

Luego etiquetaremos de acuerdo a los turnos, es decir eligiremos el contexto de acuerdo con:
   - el turno inmediatamente anterior de respuesta corta
   - los 3 turnos inmediatamente anteriores de respuesta corta
   - los 10 turnos inmediatamente anteriores de respuesta corta

In [7]:
# Ventana de 10 conversaciones
def label_with_turn(filename, frame, short_ans):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['context', 'label']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, dialect='unix')
        writer.writeheader()

        # No uso el datetime
        # Tampoco tengo en cuenta el usuario que escribe
        row = {'id_chat': 0, 'context': '', 'label': ''}
        context = []
        for idc, owner, data, hour, text in df.values:
            if is_short(text, short_ans):
                row['label'] = text
                row['context'] = ' '.join(context)
                writer.writerow(row)
                context = []

            # Descartamos la primer conversación en la lista 
            # y agregamos la última 
            if len(context) == frame:
                context = context[1:]
                context.append(text)
            else:
                context.append(text)

In [8]:
label_with_turn('turn1_short1', 1, 1)
label_with_turn('turn1_short2', 1, 2)
label_with_turn('turn1_short3', 1, 3)
label_with_turn('turn3_short1', 3, 1)
label_with_turn('turn3_short2', 3, 2)
label_with_turn('turn3_short3', 3, 3)
label_with_turn('turn10_short1', 10, 1)
label_with_turn('turn10_short2', 10, 2)
label_with_turn('turn10_short3', 10, 3)